In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
import os
import time
from datetime import datetime

In [3]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False 
opciones.add_argument('--start-maximized')
# opciones.add_argument('--incognito')     

In [4]:
e = ['ajo','albaricoque']

In [44]:
driver = "../driver/chromedriver.exe"
driver = webdriver.Chrome(driver, options=opciones)
supermarket = "eroski"
prices = []
prod = []
sup = []
links = []
names = []
frame = {}
for f in e:
    url = f"https://supermercado.eroski.es/es/search/results/?q={f}&suggestionsFilter=false"
    driver.get(url)
    time.sleep(1)
    try: #cookies
        driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
    except:
        pass
    time.sleep(.5)
    for i in range(1,15):
        try:
            prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(4) > div:nth-child(2) > p:nth-child(1) > span:nth-child(2)').text)
            names.append(driver.find_element_by_css_selector(f'#productListZone > div:nth-child({i+1}) > div > div.product-description > div.description-text > h2:nth-child(3) > a').text)
            prod.append(f)
            sup.append(supermarket)
            links.append(url)
    
        except:
            try:
                prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i+1}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(3) > div:nth-child(2) > span:nth-child(2) > span:nth-child(2)').text)
                names.append(driver.find_element_by_css_selector(f'#productListZone > div:nth-child({i+1}) > div > div.product-description > div.description-text > h2:nth-child(3) > a').text)
                prod.append(f)
                sup.append(supermarket)
                links.append(url)
            except:
                pass

frame['prices'] = prices
frame['product'] = prod
frame['supermarket'] = sup
frame['links'] = links
frame['names'] = names
scrap = pd.DataFrame(frame)
scrap.dropna(how='any',inplace=True)
if not os.path.exists(f"../mydata/scraps_{supermarket}"):
    os.makedirs(f"../mydata/scraps_{supermarket}")
    now = str(datetime.now())[:19].replace(":","_")
    scrap.to_csv(f'../mydata/scraps_{supermarket}/{now}.csv')


/tmp/ipykernel_567/2085740306.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver, options=opciones)
/tmp/ipykernel_567/2085740306.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
/tmp/ipykernel_567/2085740306.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(4) > div:nth-child(2) > p:nth-child(1) > span:nth-child(2)').text)
/tmp/ipykernel_567/2085740306.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i+1}) > div:nth-chi

In [38]:
now = str(datetime.now())[:19].replace(":","_")
print("Current Time =", now)
str(now)[:19]

Current Time = 2021-12-09 13:38:59.773519


'2021-12-09 13:38:59'

In [30]:
scrap

,prices,product,supermarket,links,names
0,"1,95",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Ajo morado IGP Las Pedroñeras EROSKI Natur, ma..."
1,"7,80 €",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Ajo blanco, malla 400 g"
2,"5,00 €",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Ajo para sazonar EROSKI, frasco 57 g"
3,"8,60 €",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Ajo ecológico EROSKI NATUR BIO, malla 150 g"
4,"8,60 €",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Caldo natural de pollo bajo en sal ANETO, brik..."
5,"1,80",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,Caldo casero de pollo bajo en sal GALLINA BLAN...
6,"1,99",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Ajo troceado FINDUS, caja 75 g"
7,"26,53 €",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Sazonador de ajo-perejil DUCROS, frasco 43 g"
8,"0,90",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Jugoso al horno pollo ajo MAGGI, sobre 36 g"
9,"1,99",ajo,eroski,https://supermercado.eroski.es/es/search/resul...,"Ajo para sazonar DUCROS, frasco 58 g"
